In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:

df = '/content/drive/My Drive/DEPI/customer/Mall_Customers.csv'
df_full = pd.read_csv(df)

In [8]:
df_full.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [9]:
df_full.shape

(200, 5)

In [10]:
columns = list(df_full.columns)
features = columns[:len(columns)-1]
class_labels = list(df_full[columns[-1]])
df = df_full[features]

In [11]:
df

,CustomerID,Genre,Age,Annual Income (k$)
0,1,Male,19,15
1,2,Male,21,15
2,3,Female,20,16
3,4,Female,23,16
4,5,Female,31,17
...,...,...,...,...
195,196,Female,35,120
196,197,Female,45,126
197,198,Male,32,126
198,199,Male,32,137


The fuzzy parameter  M




m controls the degree of fuzziness in the clustering. It determines how much a data point can belong to multiple clusters.





The value of  M  
m is always greater than 1.





The most common range is between 1.5 and 3.







 A value of   𝑚=2
m=2 is frequently used in practice.





When
𝑚=1
m=1: The FCM algorithm behaves like the K-Means algorithm, meaning each data point is assigned to exactly one cluster (hard clustering). There is no overlap, and the membership values are binary (either 0 or 1).











When
𝑚>1
m>1: The clustering becomes "fuzzy," allowing data points to have partial membership in multiple clusters. The higher the value of
𝑚
m, the softer the clustering. This means a data point can belong to multiple clusters with varying degrees of membership.

In [12]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894075 sha256=f54652be0b5faa6a9c73f683cb359397f0c0dcebf1f04fe7b21d48fbcd09e800
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [13]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

In [14]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [15]:
# Normalize the dataset
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [16]:
#The skfuzzy library's cmeans function expects the data to be in the shape of features × samples
# Transpose the data for FCM
X_transposed = X_normalized.T
X_transposed.shape


(4, 150)

The fuzzy parameter M

m controls the degree of fuzziness in the clustering. It determines how much a data point can belong to multiple clusters.

The value of M
m is always greater than 1.

The most common range is between 1.5 and 3.

A value of 𝑚=2 m=2 is frequently used in practice.

When 𝑚=1 m=1: The FCM algorithm behaves like the K-Means algorithm, meaning each data point is assigned to exactly one cluster (hard clustering).




There is no overlap, and the membership values are binary (either 0 or 1).

When 𝑚>1 : The clustering becomes "fuzzy," allowing data points to have partial membership in multiple clusters. The higher the value of 𝑚 m, the softer the clustering. This means a data point can belong to multiple clusters with varying degrees of membership.

cntr:

Description: This is an array containing the final cluster centers after the algorithm has converged.




Shape: (c, features) where c is the number of clusters, and features is the number of features in the dataset.





u:

This is the fuzzy partition matrix, which indicates the degree of membership of each data point to each cluster.



Shape: (c, n) where c is the number of clusters, and n is the number of data

 points.









d:

This is the distance matrix, which contains the distances between each data point and each cluster center.


Shape: (c, n) where c is the number of clusters, and n is the number of data points.









jm:

This is the objective function history, which records the value of the objective function at each iteration.




Type: List of objective function values.




Use: Analyzing this list helps you understand how quickly the algorithm is converging. It should decrease as the algorithm progresses.






p:

This is an array that contains the degree of participation (fuzziness) of each data point in all clusters.




It's related to the calculation of the partition matrix and is generally used internally by the algorithm.



fpc:

The fuzzy partition coefficient (FPC) is a measure of the quality of the clustering.




 It indicates how well-defined the clusters are.




Range: Values range from 0 to 1, with values closer to 1 indicating better-defined clusters.






Use: FPC can be used to evaluate and compare the clustering quality across different runs or parameter settings.

In [19]:
# Number of clusters (k = 3 for the three species of iris)
k = 3

# Fuzziness parameter indicates how much overlap you want between clusters. You've set it to 2, which is a common choice.
m = 2


#error=0.005:   This parameter specifies the stopping criterion for the algorithm.
#The algorithm will stop iterating when the change in the cluster centers is less than this value. It controls the precision of the clustering.




#init=None:This allows the algorithm to initialize the cluster centers randomly. If you provide a specific initialization, it will use that instead.



# Apply Fuzzy C-Means
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    X_transposed, c=k, m=m, error=0.005, maxiter=1000, init=None
)


The matrix u has a shape of (c, n), where:





c is the number of clusters.





n is the number of data points (samples).







Each element u[i, j] represents the membership degree of the jth data point to the ith cluster, with values ranging between 0 and 1. The sum of the membership values for a given data point across all clusters is 1.







When used with axis=0, it looks for the maximum membership value across clusters for each data point.

In [20]:


# Extract the cluster membership
cluster_membership = np.argmax(u, axis=0)

In [26]:
# Plotting the original true labels
for i in range(k):
    ax[0].scatter(X[y == i, 0], X[y == i, 1], label=iris.target_names[i])
ax[0].set_title('True Labels')
ax[0].legend()


# Plotting the FCM clustering results
for i in range(k):
    ax[1].scatter(X[cluster_membership == i, 0], X[cluster_membership == i, 1], label=f'Cluster {i+1}')
ax[1].set_title('Fuzzy C-Means Clustering')
ax[1].legend()

plt.show()

Displaying the FPC allows you to assess the quality of the clustering.


 a higher FPC value indicates better-defined clusters, making it a useful



 metric for evaluating and comparing clustering results.

In [27]:

# Display the Fuzzy Partition Coefficient (FPC) - a measure of the quality of the clustering
print(f'Fuzzy Partition Coefficient (FPC): {fpc:.4f}')


Fuzzy Partition Coefficient (FPC): 0.7065
